In [ ]:
import numpy as np
import dautil as dl
import matplotlib.pyplot as plt
import ch7util

In [ ]:
context = dl.nb.Context('sharpe_liquidity')
lr = dl.nb.LatexRenderer(chapter=7, start=3, context=context)
lr.render(r'S_a = \frac{E[R_a-R_b]}{\sigma_a} = \frac{E[R_a-R_b]}{\sqrt{\mathrm{var}[R_a-R_b]}}')

In [ ]:
def calc_metrics(ticker, ohlc):
    stock = ohlc.get(ticker)
    sp500 = ohlc.get('^GSPC')
    merged = ch7util.merge_sp500(stock, sp500)
    rets_stock = ch7util.log_rets(merged['Adj Close_stock'])
    rets_sp500 = ch7util.log_rets(merged['Adj Close_sp500'])
    stock_sp500 = rets_stock - rets_sp500
    sharpe_stock = stock_sp500.mean()/stock_sp500.std()
    avg_vol = np.log(merged['Volume_stock'].mean())

    return (sharpe_stock, avg_vol)

In [ ]:
dfb = dl.report.DFBuilder(cols=['Ticker', 'Sharpe', 'Log(Average Volume)'])

ohlc = dl.data.OHLC()

for symbol in ch7util.STOCKS:
    sharpe, vol = calc_metrics(symbol, ohlc)
    dfb.row([symbol, sharpe, vol])

df = dfb.build(index=ch7util.STOCKS)

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
_, ax = plt.subplots()
ax.scatter(df['Log(Average Volume)'], df['Sharpe'])
dl.plotting.plot_polyfit(ax, df['Log(Average Volume)'], df['Sharpe'])

dl.plotting.plot_text(ax, df['Log(Average Volume)'],
                      df['Sharpe'], ch7util.STOCKS)
ax.set_xlabel('Log(Average Volume)')
ax.set_ylabel('Sharpe')
ax.set_title('Sharpe Ratio & Liquidity')